In [2]:
import requests

def get_available_ollama_models():
    OLLAMA_URL="http://localhost:11434/api/tags"
    try:
        response = requests.get(OLLAMA_URL, timeout=5)    
        if response.status_code == 200:
            models_data = response.json()
            ret_val = sorted([model['name'] for model in models_data.get('models', [])])
        else:
            try:
                error_message = response.json()  # Try to parse error response as JSON
                error_detail = error_message.get('message', 'No detailed error message provided')
            except ValueError:  # If response isn't JSON
                error_detail = response.text  # Use plain text response if JSON parsing fails
                
            ret_val = {'status':'FAILED', 'status_code':response.status_code, 'message':error_detail} 
    except Exception as err:
        ret_val = ret_val = {'status':'FAILED', 'status_code':500, 'message':str(err)}

    return ret_val


In [3]:
get_available_ollama_models()

['deepseek-r1:1.5b',
 'deepseek-r1:32b',
 'gemma3:27b',
 'gemma3:latest',
 'llama3.2:1b',
 'llama3.3:latest',
 'nomic-embed-text:latest',
 'phi4:latest',
 'qwen2.5-ctx_32k:32b',
 'qwen2.5:32b',
 'summary-assistant:latest']

In [4]:
import ollama
client = ollama.Client()
models = client.list()
model_names = sorted([model['model'] for model in models['models']])
print(model_names)



['deepseek-r1:1.5b', 'deepseek-r1:32b', 'gemma3:27b', 'gemma3:latest', 'llama3.2:1b', 'llama3.3:latest', 'nomic-embed-text:latest', 'phi4:latest', 'qwen2.5-ctx_32k:32b', 'qwen2.5:32b', 'summary-assistant:latest']


In [5]:
model_context = []
for model_name in model_names:
    response = client.show(model_name)
    for key in response.modelinfo.keys():
        if 'context_length' in key:
            model_context.append({'model_name':model_name, 'context_size':response.modelinfo[key], 'context':key[:-15]})
            
            
#print(f"{model_name:<25} size={response.modelinfo[key]:,} \t{key:<40} ")

sorted_models = sorted(model_context, key=lambda x: (-x['context_size'], x['model_name']))



for model in sorted_models:
    print(f"{model['model_name']:<25} size={model['context_size']:,} \t{model['context']:<40} ")


deepseek-r1:1.5b          size=131,072 	qwen2                                    
deepseek-r1:32b           size=131,072 	qwen2                                    
llama3.2:1b               size=131,072 	llama                                    
llama3.3:latest           size=131,072 	llama                                    
qwen2.5-ctx_32k:32b       size=32,768 	qwen2                                    
qwen2.5:32b               size=32,768 	qwen2                                    
summary-assistant:latest  size=32,768 	qwen2                                    
phi4:latest               size=16,384 	phi3                                     
phi4:latest               size=16,384 	phi3.rope.scaling.original               
gemma3:27b                size=8,192 	gemma3                                   
gemma3:latest             size=8,192 	gemma3                                   
nomic-embed-text:latest   size=2,048 	nomic-bert                               


In [ ]:
import time 
def elapsed_time(start_time, end_time = None):
    if end_time is None:
        end_time = time.perf_counter()
    elapsed_time_seconds = end_time - start_time
    minutes, seconds = divmod(elapsed_time_seconds, 60)
    formatted_time = f"{int(minutes)} minutes and {int(seconds)} seconds"
    return formatted_time

start_time = time.perf_counter()
time.sleep(62)
formatted_time = elapsed_time(start_time)
print(formatted_time)


In [ ]:
import nltk
from nltk.corpus import stopwords
import string

# Download NLTK stopwords if not already installed
nltk.download('stopwords')

def count_words_in_file(file_path, stop_words=None):
    # Load stop words from NLTK if none are provided
    if stop_words is None:
        stop_words = set(stopwords.words('english'))
    
    # Open and read the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    
    # Split the text into words
    words = text.split()
    
    # Filter out stop words and count the remaining words
    word_counts = {}
    for word in words:
        if word not in stop_words:
            word_counts[word] = word_counts.get(word, 0) + 1
    
    return word_counts




In [ ]:
# Example usage
file_path = './youtube_summary/lex.txt'  # Replace with the path to your text file
word_counts = count_words_in_file(file_path)

# Print the word counts
word_counts_list = sorted(word_counts.items(), key=lambda item: (-item[1], item[0]))
word_counts = dict(word_counts_list)
for word, count in word_counts.items():
    print(f'{word}: {count}')

In [ ]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources if not already installed
nltk.download('punkt')

def find_two_word_combos(file_path):
    # Open and read the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Remove punctuation and convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Generate bigrams (two-word combos)
    bigrams = list(ngrams(words, 3))
    
    # Convert the bigrams to a list of string pairs (for easy readability)
    bigram_strs = [' '.join(bigram) for bigram in bigrams]
    
    return bigram_strs

In [ ]:
file_path = './youtube_summary/lex.txt'  # Replace with the path to your text file
bigrams = find_two_word_combos(file_path)

print(len(bigrams))
# Print the bigrams
for bigram in bigrams:
    print(bigram)

In [7]:
from ollama_utils import OllamaUtils

ollama_utils = OllamaUtils()
print(ollama_utils.model_names)
print(ollama_utils.model_exists('deepseek-r1:1.5b'))
print(ollama_utils.model_exists('deepseek-r1:1.5b-bad'))
print(ollama_utils.model_context_size('deepseek-r1:1.5b'))

print(ollama_utils.model_base_model('deepseek-r1:1.5b'))


['deepseek-r1:1.5b', 'deepseek-r1:32b', 'gemma3:27b', 'gemma3:latest', 'llama3.2:1b', 'llama3.3:latest', 'nomic-embed-text:latest', 'phi4:latest', 'qwen2.5-ctx_32k:32b', 'qwen2.5:32b', 'summary-assistant:latest']
True
False
131072
qwen2
